In [ ]:
#!/usr/bin/env python3

# Importing Modules
import os
import matplotlib.pyplot as plt
import pandas as pd
from pybedtools import BedTool
from math import ceil
import subprocess
import sys

In [ ]:
def concatenate_files(identifier, output_file, parent_folder):
    """
    Concatenates all .txt files containing a specific identifier within a directory tree
    and writes the combined content to a single output file.

    Parameters:
    - identifier (str): A substring to match in filenames
    - output_file (str): Path to the resulting combined file
    - parent_folder (str): Root directory to search for matching files
    """
    with open(output_file, 'w') as outfile:
        for root, dirs, files in os.walk(parent_folder):
            for file in files:
                if identifier in file and file.endswith(".txt"):
                    file_path = os.path.join(root, file)
                    with open(file_path, 'r') as infile:
                        outfile.write(infile.read())


def df_to_df(data_df):
    """
    Converts a pandas DataFrame with 'chr', 'start', and 'end' columns into a BedTool object.

    Parameters:
    - data_df (pd.DataFrame): DataFrame containing genomic intervals

    Returns:
    - BedTool object created from the DataFrame
    """
    bed_df = data_df[['chr', 'start', 'end']].reset_index(drop=True)
    bed = BedTool.from_dataframe(bed_df)
    return bed


def central_reg_from_bed(bed, center_len):
    """
    Extracts central regions of specified length from a BedTool object.

    Parameters:
    - bed (BedTool): BedTool object with genomic intervals
    - center_len (int): Length of central region to extract

    Returns:
    - BedTool object representing the central regions
    """
    extension_len = int(center_len / 2)
    bed_df = BedTool.to_dataframe(bed)
    bed_df['centre'] = ((bed_df['start'] + bed_df['end']) // 2).astype(int)

    center_bed_df = pd.DataFrame(columns=['chrom', 'start', 'end'])
    center_bed_df['chrom'] = bed_df['chrom']
    center_bed_df['start'] = (bed_df['centre'] - extension_len).astype(int)
    center_bed_df['end'] = (bed_df['centre'] + extension_len).astype(int)

    center_bed = BedTool.from_dataframe(center_bed_df)
    return center_bed


def fastaFromBed(genome_fasta, input_bed, output_fasta):
    """
    Uses bedtools to extract sequences from a genome FASTA using a BED file.

    Parameters:
    - genome_fasta (str): Path to genome FASTA file
    - input_bed (str): Path to BED file with regions of interest
    - output_fasta (str): Path to save the extracted sequences in FASTA format
    """
    command = f"bedtools getfasta -fi {genome_fasta} -bed {input_bed} -fo {output_fasta}"
    subprocess.run(command, shell=True, check=True)

In [ ]:
# Define genome and path variables
genome = 'Genome Used'
parent_folder = 'Parent folder'
genome_fasta = 'FASTA of the genome used'

# Change to the directory where parent folder is located and create a new processing directory
os.chdir(os.path.dirname(parent_folder))
os.makedirs(f"{genome}-ChARM-HiCon-Processing", exist_ok=True)
os.chdir(f"{genome}-ChARM-HiCon-Processing")

print('Step 01 of 06: Directory management Done')

In [ ]:
# Concatenate all prediction probability files into one
output_pred_prob = f'{genome}-ChARM-prediction-probabilities.txt'
concatenate_files("-Prediction-Probabilities", output_pred_prob, parent_folder)

print('Step 02 of 06: Merging prediction probabilities Done')

In [ ]:
# Read and sort the prediction probability file
pred_prob_df = pd.read_csv(output_pred_prob, sep='\t', header=None, index_col=None)
pred_prob_df.columns = ['Region', 'Class', 'Class-1-Prob', 'Class-0-Prob']
pred_prob_df = pred_prob_df.sort_values(by='Class-1-Prob', ascending=False)

# Assign deciles based on Class-1 probability
pred_prob_df_deciles = pd.qcut(pred_prob_df['Class-1-Prob'], 10, labels=False)

# Extract first and last decile DataFrames
first_decile_df = pred_prob_df[pred_prob_df_deciles == 0]
last_decile_df = pred_prob_df[pred_prob_df_deciles == 9]

print('Step 03 of 06: Decile calculation and split Done')

In [ ]:
# Convert region strings to BED format and save unmerged files
paer_unmerged = pd.DataFrame()
paer_unmerged[['chr', 'start', 'end']] = last_decile_df['Region'].str.split('-', expand=True)
paer_unmerged.to_csv(f"{genome}-HiCon-pAER-UnMerged.bed", sep="\t", index=None, header=None)

pafr_unmerged = pd.DataFrame()
pafr_unmerged[['chr', 'start', 'end']] = first_decile_df['Region'].str.split('-', expand=True)
pafr_unmerged.to_csv(f"{genome}-HiCon-pAFR-UnMerged.bed", sep="\t", index=None, header=None)

print('Step 04 of 06: pAER and pAFR UnMerged data Done')

In [ ]:
# Add BED columns to decile DataFrames
first_decile_df[['chr', 'start', 'end']] = first_decile_df['Region'].str.split('-', expand=True)
last_decile_df[['chr', 'start', 'end']] = last_decile_df['Region'].str.split('-', expand=True)

# Convert to BedTool objects
last_decile_bed = df_to_df(last_decile_df)
first_decile_bed = df_to_df(first_decile_df)

# Extract central 500bp regions
last_decile_center_bed = central_reg_from_bed(last_decile_bed, 500)
first_decile_center_bed = central_reg_from_bed(first_decile_bed, 500)

# Merge overlapping regions
last_decile_center_merge_bed = BedTool(last_decile_center_bed).sort().merge()
first_decile_center_merge_bed = BedTool(first_decile_center_bed).sort().merge()

# Identify unique regions (pAER and pAFR)
pAER = BedTool().intersect(a=last_decile_center_merge_bed, b=first_decile_center_merge_bed, wa=True, v=True)
pAFR = BedTool().intersect(b=last_decile_center_merge_bed, a=first_decile_center_merge_bed, wa=True, v=True)

# Save filtered BED files
pAER.saveas(f"{genome}-HiCon-pAER.bed")
pAFR.saveas(f"{genome}-HiCon-pAFR.bed")

print('Step 05 of 06: pAER and pAFR Merged data saving Done')

In [ ]:
# Print statistics on region counts before and after merging
print(f'Number of regions before merging for the last decile: {len(last_decile_bed)}')
print(f'Number of regions after merging for the last decile: {len(BedTool(last_decile_center_bed).sort().merge())}')
print(f'Number of regions before merging for the first decile: {len(first_decile_bed)}')
print(f'Number of regions after merging for the first decile: {len(BedTool(first_decile_center_bed).sort().merge())}')